## Import Library

In [1]:
import os
import argparse, gdal
import numpy as np
from tqdm import tqdm
from IPython.display import HTML, display
import tabulate

import keras
import keras.backend as K
from keras.models import Model, load_model
import tensorflow as tf

Using TensorFlow backend.


## 1. Land Cover Classes

In [2]:
table = [["Background", 0],
         ["Buildings", 1],
         ["Roads/Parking Lots/Driveways", 2],
         ["Planted/Darker Cropland", 3],
         ["Water", 4],
         ["Forest", 5],
         ["Harvested/Open/Bare", 6]]
display(HTML(tabulate.tabulate(table, tablefmt = 'html')))

Background,0
Buildings,1
Roads/Parking Lots/Driveways,2
Planted/Darker Cropland,3
Water,4
Forest,5
Harvested/Open/Bare,6


## Number of Land Cover Classes

In [3]:
number_of_classes = 7

color_map = [
    [52, 52, 52],
    [76, 230, 0],
    [255, 255, 115],
    [255, 190, 232],
    [204, 204, 204],
    [0, 92, 230],
    [100, 100, 100]
]

## 2. Accuracy Function

In [4]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis = 0)

## 3. Loss Function

In [5]:
class_weights = np.array([0.000001, 2, 1, 1, 1, 2, 2])
weights       = K.variable(class_weights)

def weighted_categorical_crossentropy(y_true, y_pred):
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis = -1, keepdims = True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calculate loss and weight loss
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    return loss

## 4. Save GeoTIFF Image

In [6]:
def save_image(image_data, path, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')
    
    # Set Info of Image
    height, width = image_data.shape
    dataset       = driver.Create(path, width, height, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    dataset.GetRasterBand(1).WriteArray(image_data)

    # Create Color Table
    color_table = gdal.ColorTable()
    
    for i in range(number_of_classes):
        color_table.SetColorEntry(i, tuple(color_map[i]) + (255,))
    dataset.GetRasterBand(1).SetRasterColorTable(color_table)
    dataset.GetRasterBand(1).SetNoDataValue(255)

    dataset.FlushCache()

## 5. Apply Model to Image

In [7]:
def eval_image(input_image_path, model, output_image_path):
    
    input_dataset = gdal.Open(input_image_path)
    input_image   = input_dataset.ReadAsArray().astype(np.float32)
    # print(input_image.shape)
    input_image = np.rollaxis(input_image, 0, 3)
    h, w, n     = input_image.shape
    # print(input_image.shape)
    
    model_input_height, model_input_width, model_input_channels    = model.layers[0].input_shape[1:4]
    # print(model_input_height, model_input_width, model_input_channels)
    model_output_height, model_output_width, model_output_channels = model.layers[len(model.layers) - 1].output_shape[1:4]
    # print(model_output_height, model_output_width, model_output_channels)

    padding_y = int((model_input_height - model_output_height)/2)
    padding_x = int((model_input_width - model_output_width)/2)
    assert model_output_channels == number_of_classes

    pred_lc_image = np.zeros((h, w, number_of_classes))
    # print(pred_lc_image.shape)
    mask = np.ones((h, w))
    # print(mask.shape)

    irows, icols = [],[]
    batch_size   = 16
    minibatch    = []
    ibatch       = 0
    mb_array     = np.zeros((batch_size, model_input_width, model_input_height, model_input_channels))
    # print(mb_array.shape)

    n_rows = int(h / model_output_height)
    # print(n_rows)
    n_cols = int(w / model_output_width)
    # print(n_cols)
    
    for row_idx in tqdm(range(n_rows)):
        for col_idx in range(n_cols):
            
            subimage = input_image[row_idx*model_output_height:row_idx*model_output_height + model_input_height,
                                   col_idx*model_output_width:col_idx*model_output_width + model_input_width, :] / 256.0
            # print(subimage.shape)
            
            if(subimage.shape == model.layers[0].input_shape[1:4]):
                
                mb_array[ibatch] = subimage
                ibatch += 1
                irows.append((row_idx*model_output_height + padding_y,row_idx*model_output_height + model_input_height - padding_y))
                icols.append((col_idx*model_output_width +  padding_x,col_idx*model_output_width  + model_input_width  - padding_x))

                if (ibatch) == batch_size:

                    outputs = model.predict(mb_array)
                    for i in range(batch_size):
                        r0,r1 = irows[i]
                        c0,c1 = icols[i]

                        pred_lc_image[r0:r1, c0:c1, :] = outputs[i]
                        mask[r0:r1, c0:c1] = 0

                    ibatch = 0
                    irows,icols = [],[]

    if ibatch > 0:
        outputs = model.predict(mb_array)
        for i in range(ibatch):
            r0,r1 = irows[i]
            c0,c1 = icols[i]

            pred_lc_image[r0:r1, c0:c1, :] = outputs[i]
            mask[r0:r1, c0:c1] = 0


    label_image = np.ma.array(pred_lc_image.argmax(axis=-1), mask = mask)
    save_image(label_image.filled(255), output_image_path, input_dataset.GetGeoTransform(), input_dataset.GetProjection())

## Evaluate Images in Folder

In [8]:
def evaluate(input_dir, model_path, output_dir):

    model = load_model(model_path, 
                       custom_objects = {'mean_iou': mean_iou, 'weighted_categorical_crossentropy': weighted_categorical_crossentropy})

    for root, dirs, files in os.walk(input_dir):
        if not files: continue

        for f in files:
            pth     = os.path.join(root,f)
            out_pth = os.path.join(output_dir,f.split('.')[0]+'_C.tif')
            eval_image(pth, model, out_pth)
            print('saved result to ' + out_pth)

## 6. Set Parameters

In [11]:
input_image_dir  = r'D:\land_cover\data\raw\image'
model            = r'D:\land_cover\models\model_epochs_05.h5'
output_image_dir = r'D:\land_cover\data\results'

## Run Script Keras

In [12]:
evaluate(input_image_dir, model, output_image_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:15<00:00,  2.15it/s]


saved result to D:\land_cover\data\results\m_3008718_ne_16_1_20171024_C.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:12<00:00,  2.21it/s]


saved result to D:\land_cover\data\results\m_3008718_nw_16_1_20171024_C.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:12<00:00,  2.24it/s]


...

saved result to D:\land_cover\data\results\m_3408602_nw_16_1_20170711_C.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:11<00:00,  2.53it/s]


saved result to D:\land_cover\data\results\m_3408602_se_16_1_20170711_C.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:12<00:00,  2.52it/s]


saved result to D:\land_cover\data\results\m_3408602_sw_16_1_20170711_C.tif
